<a href="https://colab.research.google.com/github/ivenk/ma_dl/blob/master/ma_keygen_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms

In [0]:
from google.colab import files

In [0]:
!ls

__MACOSX  sample_data


In [0]:
data = files.upload()

Saving lfw.out.zip to lfw.out.zip


KeyboardInterrupt: ignored

In [0]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
downloaded = drive.CreateFile({'id':"1qQw8ecr4L7zk36ylac-ak7znh-1lIOfg"})   # replace the id with id of file you want to access
downloaded.GetContentFile('lfw3.out.zip') 

In [0]:
!ls

adc.json  lfw3.out.zip	__MACOSX  sample_data


In [0]:
!unzip lfw3.out.zip

Archive:  lfw3.out.zip
   creating: lfw.out/
   creating: lfw.out/182694872172212169954038859952714876710/
  inflating: lfw.out/182694872172212169954038859952714876710/0_0.png  
  inflating: lfw.out/182694872172212169954038859952714876710/0_1.png  
  inflating: lfw.out/182694872172212169954038859952714876710/0_2.png  
   creating: lfw.out/103459878855177950767319593705665515565/
  inflating: lfw.out/103459878855177950767319593705665515565/3_1.png  
  inflating: lfw.out/103459878855177950767319593705665515565/1_2.png  
  inflating: lfw.out/103459878855177950767319593705665515565/3_0.png  
  inflating: lfw.out/103459878855177950767319593705665515565/3_2.png  
  inflating: lfw.out/103459878855177950767319593705665515565/1_0.png  
  inflating: lfw.out/103459878855177950767319593705665515565/1_1.png  
  inflating: lfw.out/103459878855177950767319593705665515565/0_0.png  
  inflating: lfw.out/103459878855177950767319593705665515565/2_2.png  
  inflating: lfw.out/10345987885517795076731959370

In [0]:
class MultiClass(nn.Sequential):
    
    def __init__(self, input_dim=512, hidden_layers=5, output_dim=128):
        super(MultiClass, self).__init__()

        layers = [
            ("cf1" , nn.Linear(input_dim, input_dim)),
            ("r1"  , nn.ReLU()),
            ("cf1" , nn.Linear(input_dim, input_dim)),
            ("r1"  , nn.ReLU()),
            ("cf1" , nn.Linear(input_dim, input_dim)),
            ("r1"  , nn.ReLU()),
            ("cf1" , nn.Linear(input_dim, input_dim)),
            ("r1"  , nn.ReLU()),
            ("cf1" , nn.Linear(input_dim, output_dim)),
        ]
        
        [self.add_module(name, layer) for (name, layer) in layers]
        
    def classifier(self,z):
        return nn.functional.softmax(self(z))

In [0]:
class MultiClass(nn.Sequential):
    
    def __init__(self, input_dim=512, hidden_layers=5, output_dim=128):
        super(MultiClass, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l3 = nn.Linear(input_dim, input_dim)
        self.l4 = nn.Linear(input_dim, input_dim)
        self.l5 = nn.Linear(input_dim, output_dim)
    

 #   def classifier(self,z):
  #      return nn.functional.softmax(self(z))

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))       
        x = F.relu(self.l4(x))
        x = F.relu(self.l5(x))
        return torch.sigmoid(x)


In [0]:
model = MultiClass()

In [0]:
def collate_fn(x):
    return x[0]


In [0]:
dataset = datasets.ImageFolder('./lfw.out/')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn)

In [0]:
loader?

In [0]:
# Load the pretrained model
resnet = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = resnet._modules.get('avgpool')

In [0]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [0]:
def get_vector(img):
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))    
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)    

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.squeeze())    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    resnet(t_img)    
    # 7. Detach our copy function from the layer
    h.remove()    
    # 8. Return the feature vector
    return my_embedding

In [0]:
def get_target_vector(target):
  data = []

  binary = f'{int(target):0128b}'
  for c in binary:
    data.append(int(c))

  return torch.tensor(data)

In [0]:
from torch.autograd import Variable

In [0]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [0]:
loss_func = nn.NLLLoss()

# Adam optimizer with learning rate 0.1 and L2 regularization with weight 1e-4.
optimizer = torch.optim.Adam(model.parameters(),lr=0.1,weight_decay=1e-4)




In [0]:
epochs = 5
for e in range(epochs):
  loss_epoch = 0
  for index, (data, target) in enumerate(loader):
    # prep
    data_t   = get_vector(data)
    target_t = get_target_vector(dataset.idx_to_class[target])

    data_v   = Variable(data_t, requires_grad=False)
    target_v = Variable(target_t, requires_grad=False)

    # forward
    pred = model.forward(data_v)

    optimizer.zero_grad()
    # calculate loss
    
    loss = loss_func(pred, target_v.float())
    loss_epoch = loss_epoch + loss

    loss.backward()
    optimizer.step()

    if (index % 100) == 0:
      print(str(index))

  print(loss_epoch)



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

KeyboardInterrupt: ignored